FIRST DRAFT: I won't think much of the markdown until I'm done.

This is my Fantasy Hockey Analyzer. The purpose of this project is to predict the fantasy hockey output of individual skaters based on stats from previous years.

In [2]:
#Import block
import os
import numpy as np
import pandas as pd

In [3]:
#The current working directory is the main repository directory; these lines set the path to where the data is
path = os.getcwd()
data_path = path + '\\data'

#This array makes it easier to format the rotowire data
rw_labels = ["name", "Team", "Pos", "Games", "Goals", "Assists", "Pts", "+/-", "PIM", "SOG", "GWG", "PP_Goals", "PP_Assists", "SH_Goals", "SH_Assists", "Hits", "Blocked_Shots"]

#This is the breakdown of how many fantasy points a player gets for each category
points_dict = {"Goals":5, "Assists":3, "+/-":1.5, "PIM":-0.25, "PP_Goals":4, "PP_Assists":2, "SH_Goals":6, "SH_Assists":4, "Faceoffs_Won":0.25, "Faceoffs_Lost":-0.15, "Hits":0.5, "Blocked_Shots":0.75 }


In [4]:
#I have data from the 2010-2011 season through the 2021-2022 season.
#By the end of this block, there will be 12 seasons-worth of data in the "data" variable
yearly_player_data = []

for i in range(2010, 2022):
    new_data = []
    
    #Imports the rotowire and moneypuck datasets from the selected year into rdf and mdf
    rdf = pd.read_csv(data_path + '\\rotowire_data\\rotowire{}.csv'.format(str(i)))
    mdf = pd.read_csv(data_path + '\\moneypuck_data\\moneypuck{}.csv'.format(str(i)))
    
    #Formats the rotowire data
    rdf.set_axis(rw_labels, axis=1, inplace=True)
    rdf.drop(index=rdf.index[0], axis=0, inplace=True)
    
    #The Moneypuck data has information about 5-on-5, 5-on-4, 4-on-5, other, and all.
    #For this project I'm just focused on "all" since I suspect it'll give me the best results.
    mdf = mdf[mdf["situation"] == "all"]
    
    #Merges the rotowire and moneypuck dataframes
    new_data = pd.merge(rdf, mdf, on="name")
    
    #Changes the name of a few columns in the new dataframe
    new_data = new_data.rename(columns={"name":"Name","faceoffsWon":"Faceoffs_Won","faceoffsLost":"Faceoffs_Lost"})
    
    #This section calculates each player's total fantasy output for that year
    cols = new_data.columns
    fant_points = [0 for i in range(len(new_data))]
    for i in range(len(new_data)):
        for j in range(len(new_data.iloc[i])):
            mult = points_dict.get(cols[j], 0)
            if mult != 0:
                fant_points[i] += mult*int(new_data.iloc[i, j])
    
    #Adds the players' fantasy points to the new_data dataframe
    new_data["Fantasy_Points"] = fant_points
    
    #Adds new_data to the "data" array
    yearly_player_data.append(new_data)
                
    



At this point I have player data for each individual year. Now, I want to turn this data into data that I can put into a machine learning model. 

At this moment, I will only use data for players that have been in the league for 3 years, such that the model will be predicting fantasy output for their fourth year.

In [5]:
yearly_player_data[1]

,Name,Team,Pos,Games,Goals,Assists,Pts,+/-,PIM,SOG,...,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts,Fantasy_Points
0,Steven Stamkos,TB,C,82,60,37,97,7,66,303,...,159.11,2328.0,3268.0,0.0,0.0,0.0,0.0,0.0,0.0,600.40
1,Evgeni Malkin,PIT,C,75,50,59,109,18,70,339,...,125.59,2592.0,2685.0,0.0,0.0,0.0,0.0,0.0,0.0,622.25
2,Marian Gaborik,FA,RW,82,41,35,76,15,34,276,...,142.35,2632.0,3279.0,0.0,0.0,0.0,0.0,0.0,0.0,447.20
3,James Neal,FA,RW,80,40,41,81,6,87,329,...,141.62,2982.0,2980.0,0.0,0.0,0.0,0.0,0.0,0.0,470.85
4,Alex Ovechkin,WAS,LW,78,38,27,65,-8,26,303,...,138.04,2497.0,3148.0,0.0,0.0,0.0,0.0,0.0,0.0,463.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,Reilly Smith,VGK,RW,3,0,0,0,-3,2,2,...,7.31,140.0,139.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.75
864,Jeremy Welsh,FA,C,1,0,0,0,0,4,2,...,1.57,38.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.15
865,Mattias Ekholm,NSH,D,2,0,0,0,-1,0,1,...,5.49,69.0,108.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75
866,Mika Zibanejad,NYR,C,9,0,1,1,-3,2,12,...,25.51,428.0,400.0,0.0,0.0,0.0,0.0,0.0,0.0,12.55


In [6]:
ml_data = []
for i in range(2013, 2022):
    curr = yearly_player_data[i-2010]
    for index, row in curr.iterrows():
        for j in range(3):
            if 

Alex Ovechkin
Corey Perry
Joe Pavelski
Max Pacioretty
Phil Kessel
Tyler Seguin
Sidney Crosby
Chris Kunitz
Patrick Sharp
Jamie Benn
Alexander Steen
Patrick Marleau
Ryan Johansen
Jeff Skinner
Ryan Getzlaf
Patrice Bergeron
Jason Pominville
Marian Hossa
Martin St. Louis
Jarome Iginla
James van Riemsdyk
Patrick Kane
Wayne Simmonds
Zach Parise
Anze Kopitar
Blake Wheeler
Jordan Eberle
David Perron
Claude Giroux
Jonathan Toews
Ryan O'Reilly
Gustav Nyquist
Taylor Hall
Kyle Okposo
James Neal
David Backes
Jeff Carter
Thomas Vanek
Rick Nash
Michael Cammalleri
Kyle Turris
Gabriel Landeskog
Jaden Schwartz
Adam Henrique
Frans Nielsen
Paul Stastny
Troy Brouwer
Brad Marchand
Steven Stamkos
Ryan Kesler
Valtteri Filppula
Clarke MacArthur
Antoine Vermette
Jaromir Jagr
Milan Lucic
John Tavares
Joel Ward
Craig Smith
Tyler Johnson
Nathan MacKinnon
Ondrej Palat
Matt Moulson
Jakub Voracek
Logan Couture
Bryan Little
Matt Duchene
Shane Doan
Bobby Ryan
Evgeni Malkin
Andrew Ladd
Shea Weber
Jason Spezza
Alexander S

Vladislav Namestnikov
Boone Jenner
Rickard Rakell
Jared Spurgeon
Taylor Beck
Ryan Spooner
Matt Read
Ryan Garbutt
Marko Dano
Seth Jones
Rasmus Ristolainen
Markus Granlund
Leo Komarov
Morgan Rielly
Matt Dumba
Martin Hanzal
Michael Grabner
Brent Seabrook
James Wisniewski
Jack Johnson
Brad Richardson
Vincent Lecavalier
Zdeno Chara
Daniel Briere
Tomas Fleischmann
Scottie Upshall
Jussi Jokinen
Marcus Foligno
Matt Martin
Matt Irwin
Matt Fraser
Mikael Granlund
Riley Nash
Kevin Shattenkirk
Ryan McDonagh
Alexander Edler
Joe Colborne
Matt Hendricks
Artem Anisimov
Tyler Myers
Drew Doughty
Alex Pietrangelo
David Booth
Cody McLeod
Mark Letestu
Derek Dorsett
Cal Clutterbuck
Cam Fowler
Marcus Kruger
Ben Smith
Jeff Petry
Matt Stajan
Brooks Laich
Tuomo Ruutu
Chris Kelly
Jay McClement
Marek Zidlicky
Matt Cullen
Scott Gomez
Jason Chimera
Eric Nystrom
Johan Franzen
Andrej Meszaros
Chris Thorburn
James Sheppard
Cody Franson
David Krejci
Jacob Trouba
Hampus Lindholm
Vincent Trocheck
Erik Haula
Brad Malone
An

Devin Shore
Jujhar Khaira
Adam Pelech
Scott Kosmachuk
Esa Lindell
Chandler Stephenson
Erik Gustafsson
Brett Kulak
Trevor Carrick
Taylor Leier
Christoph Bertschy
Charles Hudon
Brendan Woods
Seth Griffith
Tyler Wotherspoon
Andrey Pedan
Xavier Ouellet
Brett Ritchie
Daniel Catenacci
Michael Paliotta
Alexandre Grenier
Kyle Rau
Scott Harrington
Stuart Percy
Stefan Noesen
Jamie Oleksiak
Ryan Murphy
Mark McNeill
Brady Skjei
Slater Koekkoek
Marek Hrivik
Yanni Gourde
Brad Hunt
Derrick Pouliot
Griffin Reinhart
Cole Schneider
Brian O'Neill
Turner Elson
Drew Shore
Sean Collins
Chris Brown
Jeremy Welsh
Tommy Cross
Brian Dumoulin
Fredrik Claesson
Max Friberg
Josh Archibald
Tyler Graovac
Brody Sutter
Dean Kukan
Noel Acciari
Markus Hannikainen
Kyle Baun
Dennis Everberg
Kasperi Kapanen
Vojtech Mozik
Oliver Kylington
Dominik Simon
Sergei Plotnikov
Ryan Johnston
Casey Nelson
Brandon Tanev
Pavel Zacha
Mikko Rantanen
Aaron Ness
Taylor Fedun
Harry Zolnierczyk
Derek Grant
Paul Thompson
Philip Samuelsson
Ben S

Brian Lashoff
Justin Falk
Teddy Purcell
Adam Pardy
Cal O'Reilly
Mason Raymond
Tom Sestito
Jared Boll
Cody Bass
Nick Schultz
Brooks Orpik
Zbynek Michalek
Jakub Kindl
Nicklas Grossmann
Marc Methot
Clarke MacArthur
John-Michael Liles
Matt Carle
Gabriel Bourque
Mitch Callahan
Eric Gelinas
Alex Biega
Kevin Connauton
Andrew Desjardins
Luke Gazdic
Carl Gunnarsson
Simon Despres
Erik Condra
Brian Strait
Petter Granberg
Julian Melchiori
Patrik Nemeth
Quinton Howden
Corey Tropp
Emerson Etem
Cody Goloubef
Garrett Mitchell
David Warsofsky
Erik Burgdoerfer
Anton Rodin
Byron Froese
Nicolas Deslauriers
Linden Vey
Mark McNeill
Duncan Siemens
Ryan Murphy
Dalton Prout
Tim Heed
Brandon Davidson
Anthony Bitetto
Taylor Beck
Mark Fayne
Aaron Ness
Derek Grant
Taylor Fedun
Paul Thompson
Zach Redmond
Justin Dowling
Ben Street
Tyler Wotherspoon
Miikka Salomaki
Alexandre Grenier
Ty Rattie
Rocco Grimaldi
Joe Morrow
Nicklas Jensen
Cole Schneider
Max McCormick
Joseph LaBate
Dillon Simpson
Robbie Russo
Andy Andreoff


Jeff Carter
Mathieu Joseph
Conor Garland
Nolan Patrick
Pavel Zacha
Derek Ryan
Kevin Fiala
Aaron Ekblad
Sam Bennett
Dominik Kahun
Melker Karlsson
Adrian Kempe
Brendan Lemieux
Miro Heiskanen
Casey Mittelstadt
Austin Wagner
Jordan Greenway
Vince Dunn
Drake Caggiula
Troy Brouwer
Tyler Ennis
Colin Wilson
Victor Hedman
Mats Zuccarello
Charlie Coyle
Scott Laughton
Pontus Aberg
Andre Burakovsky
Shea Theodore
Ryan Hartman
Phillip Danault
Chris Wagner
Brandon Pirri
Adam Lowry
Markus Granlund
Josh Archibald
Matt Dumba
Vladislav Namestnikov
Artturi Lehkonen
Andrew Copp
Damon Severson
Esa Lindell
Jaden Schwartz
Justin Faulk
Taylor Hall
Matt Calvert
Kyle Clifford
Magnus Paajarvi
Jordan Staal
Loui Eriksson
Tyson Jost
Filip Chytil
Garnet Hathaway
Zach Werenski
Lawson Crouse
Christian Fischer
Danton Heinen
Ondrej Kase
Jesperi Kotkaniemi
Warren Foegele
Lucas Wallmark
Ryan Donato
Alexander Steen
Frans Nielsen
Alexander Edler
Calle Jarnkrok
Devin Shore
Brock McGinn
Colton Parayko
Miles Wood
Darnell Nurse


Peter Cehlarik
Jean-Sebastien Dea
Tyler Lewington
Anton Blidh
John Gilmour
Luke Johnson
Jonny Brodzinski
Nick Jensen
Samuel Morin
Marko Dano
Gustav Olofsson
Carl Dahlstrom
Justin Bailey
Nikolay Goldobin
Joel Hanley
Ryan MacInnis
Andreas Englund
John Quenneville
Chase De Leo
William Lagesson
Jake Walman
Mark Friedman
Austin Poganski
Clark Bishop
Anton Lindholm
Karl Alzner
Mark Letestu
Justin Abdelkader
Colin Wilson
Jason Demers
Brian Lashoff
Dan Hamhuis
Trevor Daley
Jonathan Ericsson
Kris Russell
Roman Polak
Chris Stewart
Kevin Gravel
Dalton Prout
Zach Trotman
Brandon Pirri
Michael Sgarbossa
Michael Chaput
Sven Baertschi
Ben Street
Cory Conacher
Brandon Davidson
Anthony Bitetto
Aaron Ness
Brian Gibbons
Kevin Connauton
Steven Kampfer
Matt Irwin
Dalton Smith
Derek Forbort
Dylan McIlrath
Mark Barberio
Alex Biega
Gabriel Dumont
Jordan Szwarz
David Savard
Christian Wolanin
Alexandre Carrier
Colton White
Anthony Richard
Niko Mikkola
Christian Jaros
Cameron Hughes
Brett Seney
Kole Sherwood
Mas

Andrew Mangiapane
Trevor Moore
Kris Letang
Adam Pelech
Chris Kreider
Sidney Crosby
Anthony Duclair
Andrei Svechnikov
Erik Haula
Garnet Hathaway
Sebastian Aho
Sebastian Aho
Sebastian Aho
Sebastian Aho
Nikolaj Ehlers
Conor Garland
Adam Fox
Darnell Nurse
Ryan Lindgren
Alexander Edler
Leon Draisaitl
Robert Thomas
Marco Scandella
Olli Maatta
Jason Robertson
Brad Marchand
Joel Eriksson Ek
Ilya Mikheyev
Blake Coleman
Mikael Backlund
Morgan Rielly
Colton Parayko
Brenden Dillon
Roope Hintz
J.T. Miller
Andrew Copp
Ryan Strome
Ondrej Palat
Anthony Cirelli
Brandon Montour
Blake Lizotte
Martin Fehervary
Erik Gudbranson
Timothy Liljegren
Luke Schenn
Jon Merrill
Ryan McDonagh
Ian Cole
Duncan Keith
Tyler Myers
Phillip Danault
Paul Stastny
Mattias Ekholm
Justin Holl
Jake Guentzel
David Pastrnak
Nazem Kadri
Tom Wilson
Roman Josi
John Carlson
Pierre Engvall
Barclay Goodrow
Dante Fabbro
Filip Forsberg
Andre Burakovsky
Nicolas Roy
Frederick Gaudreau
Mathieu Joseph
David Kampf
Mike Matheson
Teddy Blueger
Ra